In [1]:
import os


In [2]:
!pip install python-box

In [4]:
%pwd

'c:\\Users\\mayuo\\OneDrive\\Documents\\Machine Learning by Abraham\\MLOps wine'

In [2]:
os.chdir('../')

#### We are updating the data ingestion and this will be in the entity folder

In [5]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) # frozen means it is immutable. 
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

#### This will be an update in out configuration.py 

In [6]:
from src.MLOps_wine.constants import *
from src.MLOps_wine.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    # we create the constructor first. 
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        # create instanses for each using commons we have created. 
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # creating the directories
        create_directories([self.config.artifacts_root])

    # this function helps create the paths?? 
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

### Now we create the data ingestion component.

In [3]:
import os
import urllib.request as request
import zipfile
from src.MLOps_wine import logger
from src.MLOps_wine.utils.common import get_size

In [9]:
## we create the constructor first
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    ## this can also be get for AWS, or URL or even your local directory.
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

#### We need to update the pipeline
- This is the flow of how you call functions

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-28 17:33:39,712: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-28 17:33:39,716: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-28 17:33:39,719: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-28 17:33:39,721: INFO: common: created directory at: artifacts]
[2024-04-28 17:33:39,722: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-28 17:33:40,267: INFO: 463201800: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "523bdbffd2a475b493532f3591e863af56ce829d4f75504c8c40a1f834f97514"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D0AC:1BAFE0:116803B:139539D:662EEAE1
Accept-Ranges: bytes
Date: Mon, 2